<a href="https://colab.research.google.com/github/jeguns/EP7173/blob/main/Unidad%2003/03_Tratamiento_de_valores_perdidos_descripci%C3%B3n_y_mecanismos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalación de paquetes**

In [ ]:
pip install pandas==1.5.3 &> /dev/null

In [ ]:
import pandas as pd

In [ ]:
print(pd.__version__)

In [ ]:
pip install rpy2 &> /dev/null

In [ ]:
import rpy2

In [ ]:
print(rpy2.__version__)

# **Ejemplo**

Archivo: 03 - Datos - 1.xlsx

Conjunto de datos de 20 observaciones y 6 columnas:
* IQ: Coeficiente intelectual del trabajador
* JP_COM: Desempeño laboral (columna completa)
* JP_MCAR: Desempeño laboral (valores perdidos acorde a un mecanismo MCAR)
* JP_MAR: Desempeño laboral (valores perdidos acorde a un mecanismo MAR)
* JP_MNAR: Desempeño laboral (valores perdidos acorde a un mecanismo MNAR)
* AREA: Área a la que pertenece el trabajador

In [ ]:
datos = pd.read_csv('03 - datos_diabetes.csv',  sep = ";")
datos.info()

La lectura inicial de los datos ha sido incorrecta en cuanto a los tipos de variables y la cantidad de datos perdidos reconocidos, que en este caso son 0. Entonces, al momento de realizar la lectura, se puede declarar cuál es (o cuáles son) la manera de identificar un dato perdido.

In [ ]:
datos = pd.read_csv('03 - datos_diabetes.csv', sep = ";", na_values=["-"])
datos.info()

In [ ]:
new_column_names = {
    'Ingresos Anuales': 'Ingresos',
    'Horas de Sueño por Noche': 'Sueno',
    'Historial de Enfermedades Crónicas': 'Enfermedades',
    'Nivel de Estrés': 'Estres'
}

datos = datos.rename(columns=new_column_names)

# Identificación de valores perdidos

En Python, nan significa Not a Number, es decir es un valor perdido de tipo numérico (un valor numérico indefinido). Se utiliza principalmente para representar valores faltantes o no definidos en cálculos matemáticos y científicos. Por otro lado, None es de tipo object, se utiliza para representar la ausencia de un valor en cualquier tipo de objeto.

In [ ]:
import numpy as np
a = np.nan
a-3, a*2, type(a)

In [ ]:
b = None
# b*2 # Quite el # inicial y ejecute

¿Dónde están los valores perdidos?

In [ ]:
datos.head(5).isna()

  False hace referencia a una celda con dato, mientras que True se refiere a una celda vacía.

In [ ]:
datos.head(5).isnull()

In [ ]:
datos.head(5).notna()

True hace referencia a un dato completo, mientras que Falso, a un valor perdido.

¿Cuántos valores perdidos / completos hay? ¿En qué columnas o filas están los datos perdidos?

In [ ]:
datos.isna().any() # Para cada una de las columnas, devuelve True si por lo menos hay un valor faltante, y False si toda la columna está completa

In [ ]:
datos.isna().sum() # Suma la cantidad de celdas nulas o sin datos por columna

In [ ]:
datos.count()

In [ ]:
datos.describe()

In [ ]:
datos.describe(include = "all")

¿Cómo podemos visualizar los datos perdidos?

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt
msno.bar(datos, figsize=(15, 8), fontsize=12, color='steelblue')
plt.title('Frecuencia de valores completos en el dataset', fontsize=16)
plt.show()

In [ ]:
msno.matrix(datos) # permite ver los missings como espacios en blanco
plt.title('Ubicación de los valores perdidos en el dataset', fontsize=16)
plt.show()

In [ ]:
msno.dendrogram(datos)
plt.title('Valores perdidos en el dataset', fontsize=16)
plt.show()

In [ ]:
msno.heatmap(datos)
plt.title('Asociación de pérdida de valores', fontsize=16)
plt.show()
# Varía entre -1 (si una variable está presente, la otra está ausente, y viceversa)
# 0 (la presencia o ausencia de datos de una variable no se relaciona con la otra) a
# 1 (si una variable está presente, la otra también, o si una está ausente la otra también).

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%R -i datos

In [ ]:
%R datos |> head(5)

In [ ]:
%R install.packages("naniar")

In [ ]:
%%R
library(naniar)
datos |> gg_miss_upset()

# **Comparación MCAR vs MAR**

## Revisión exploratoria

In [ ]:
datos[datos['Ingresos'].isnull()].describe(), datos[datos['Ingresos'].notnull()].describe()

In [ ]:
datos[datos['Sueno'].isnull()].describe(), datos[datos['Sueno'].notnull()].describe()

In [ ]:
datos[datos['Estres'].isnull()].describe(), datos[datos['Estres'].notnull()].describe()

## RBtest

Test de verificación de tipo de valores perdidos usando el enfoque de regresión

La respuesta puede dar:

- 0: variable(s) con datos MCAR

- 1: variable(s) con datos MAR

- -1: variable(s) con datos completos

In [ ]:
%R install.packages("RBtest")

In [ ]:
%%R
library(RBtest)
library(dplyr)

In [ ]:
%%R
datos |> RBtest()

## Test MCAR de Little

$H_0$: Los datos siguen un patrón MCAR

$H_1$: Los datos no siguen un patrón MCAR

In [ ]:
%%R
datos |> select(Edad,Sexo,Enfermedades) |> mcar_test()

El pvalor nos sirve para tomar una decisión respecto a la **hipótesis ...**. La regla de decisión es: **Si ..., entonces ...**

Recordar que todo pvalor toma valores entre ... y ....

Entonces, en este caso pvalor = ..., por lo tanto se debería .... (los datos siguen un patrón ...). Sin embargo, los datos están completos. Por lo tanto, solo se debe usar esta prueba cuando hay al menos un dato perdido en el data frame.

In [ ]:
%%R
datos |> mcar_test()

El pvalor (0.169) es alto (mayor que los niveles de significancia usuales), por lo tanto se ... la hipótesis nula de mecanismo MCAR.

In [ ]:
%%R
datos |> select(Edad,Sexo,Enfermedades,Estres) |> mcar_test()

El pvalor (0.0066) es muy pequeño (menor a los niveles de significancia 0.01, 0.05, 0.10). Entonces se ...  la hipótesis nula, por lo tanto los datos ... siguen un patrón MCAR.

## Prueba de independencia - Prueba Chi Cuadrado

Cuando se tiene interés en cruzar el mecanismo de datos perdidos en una variable con otra (categórica) de interés (completa).

$H_0:$ El mecanismo de datos perdidos en [variable con datos perdidos] y la [variable de interés completa] son independientes

$H_1:$ El mecanismo de datos perdidos en [variable con datos perdidos] y la [variable de interés completa] **no** son independientes

**El tamaño de muestra debe ser grande, de modo que la frecuencia esperada en cada celda sea superior a 5.**

Solo a modo de ejemplo (ya que el tamaño de muestra es pequeño), para el conjunto de datos en estudio, se analizará la asociación del mecanismo de datos perdidos con la variable ÁREA.

Asumiendo $\alpha=0.10$

In [ ]:
%%R
chisq.test(table(as_shadow(datos)$Ingresos_NA, datos$Sexo))

In [ ]:
%%R
chisq.test(table(as_shadow(datos)$Ingresos_NA, datos$Enfermedades))

In [ ]:
%%R
chisq.test(table(as_shadow(datos)$Sueno_NA, datos$Enfermedades))